In [2]:
import wmi
import os
import time
import logging
from typing import List, Dict
from datetime import datetime

In [3]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Initialize storage
battery_data: List[Dict] = []

In [5]:

def initialize_wmi():
    try:
        return wmi.WMI()
    except Exception as e:
        logger.error(f"Failed to initialize WMI: {e}")
        return None

def collect_battery_data():
    c = initialize_wmi()
    if not c:
        logger.error("Could not initialize WMI client")
        return
    
    try:
        while len(battery_data) < 300:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            batteries = c.Win32_Battery()
            if not batteries:
                logger.warning(f"[{current_time}] No battery found")
                time.sleep(60)  # 1 minutes
                continue
                
            for battery in batteries:
                record = {
                    'battery_percentage': getattr(battery, 'EstimatedChargeRemaining', None),
                    'voltage': getattr(battery, 'DesignVoltage', None),
                    'temperature': getattr(battery, 'Temperature', None),
                    'full_charge_capacity': getattr(battery, 'FullChargeCapacity', None),
                    'time': time.time(),
                    'timestamp': current_time
                }
                battery_data.append(record)
                logger.info(f"[{current_time}] Collected battery data: {record}")
            
            time.sleep(60)  # 1 minutes
            
    except Exception as e:
        logger.error(f"Error collecting battery data: {e}")


In [6]:
def save_battery_data():
    try:
        # Create data directory if it doesn't exist
        os.makedirs('data', exist_ok=True)
        
        # Convert battery data to DataFrame
        df = pd.DataFrame(list(battery_data))
        
        # Save to CSV
        filename = 'data/battery_data.csv'
        df.to_csv(filename, index=False)
        logger.info(f"Battery data saved to {filename}")
        
    except Exception as e:
        logger.error(f"Error saving battery data: {e}")


In [7]:

if __name__ == "__main__":
    collect_battery_data()
    save_battery_data()

INFO:__main__:[2024-11-19 18:57:33] Collected battery data: {'battery_percentage': 53, 'voltage': '11165', 'temperature': None, 'full_charge_capacity': None, 'time': 1732022853.2035682, 'timestamp': '2024-11-19 18:57:33'}
INFO:__main__:[2024-11-19 18:58:33] Collected battery data: {'battery_percentage': 53, 'voltage': '12043', 'temperature': None, 'full_charge_capacity': None, 'time': 1732022913.418907, 'timestamp': '2024-11-19 18:58:33'}
INFO:__main__:[2024-11-19 18:59:33] Collected battery data: {'battery_percentage': 54, 'voltage': '12130', 'temperature': None, 'full_charge_capacity': None, 'time': 1732022973.691991, 'timestamp': '2024-11-19 18:59:33'}
INFO:__main__:[2024-11-19 19:00:33] Collected battery data: {'battery_percentage': 55, 'voltage': '12176', 'temperature': None, 'full_charge_capacity': None, 'time': 1732023033.9123847, 'timestamp': '2024-11-19 19:00:33'}
INFO:__main__:[2024-11-19 19:01:33] Collected battery data: {'battery_percentage': 55, 'voltage': '12243', 'temper